<a href="https://colab.research.google.com/github/pablomiralles22/class-CV-computer-vision/blob/main/Convolution_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch torchvision matplotlib pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

## 📦 Data Loading
In this section, we load the MNIST dataset. MNIST is a classic dataset of handwritten digits (0–9). Each image is 28x28 pixels in grayscale. We'll use this data to train a simple image classification model.

In [3]:
# --- Data ---
transform = transforms.ToTensor()

train_dataset = datasets.MNIST('.', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = datasets.MNIST('.', train=False, transform=transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 11.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 331kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.16MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.55MB/s]


Let's explain the code step by step:

```python
transform = transforms.ToTensor()
```

* Converts PIL images or NumPy arrays to PyTorch tensors.
* Scales pixel values from `[0, 255]` to `[0.0, 1.0]`.

```python
train_dataset = datasets.MNIST('.', train=True, download=True, transform=transform)
```

* Loads the **training set** of the MNIST dataset.
* Stores data in the current directory (`'.'`).
* Downloads the dataset if not already present.
* Applies `ToTensor()` transformation.

```python
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
```

* Wraps the training dataset in a data loader.
* Loads data in **mini-batches of 64**.
* Shuffles data every epoch for better training.

```python
val_dataset = datasets.MNIST('.', train=False, transform=transform)
```

* Loads the **validation (test) set** of MNIST.
* No shuffling or downloading since it uses the test split.

```python
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
```

* Loads the validation data in batches of 64.
* **No shuffling**, preserving original order (typical for evaluation).


## 🧠 Define the Neural Network
Here, we define our neural network using PyTorch's nn.Module. It's a simple Convolutional Neural Network (CNN) with three convolutional layers followed by a final classification layer. CNNs are especially good for image recognition tasks.

In [4]:
# --- Model ---
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2),   # [B, 1, 28, 28] -> [B, 16, 13, 13]
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2),  # [B, 16, 13, 13] -> [B, 32, 6, 6]
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2),  # [B, 32, 6, 6] -> [B, 64, 2, 2]
            nn.ReLU(),
        )
        self.classifier = nn.Linear(64 * 2 * 2, 10)  # Flatten: [B, 256] -> [B, 10]

    def forward(self, x):
        x = self.net(x)
        x = x.flatten(start_dim=1)
        return self.classifier(x)

```python
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
```

* Defines a custom neural network class inheriting from `nn.Module`.

```python
        self.net = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2),   # [B, 1, 28, 28] -> [B, 16, 13, 13]
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2),  # [B, 16, 13, 13] -> [B, 32, 6, 6]
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2),  # [B, 32, 6, 6] -> [B, 64, 2, 2]
            nn.ReLU(),
        )
```

* A **convolutional feature extractor** with 3 layers:

  * Each `Conv2d` halves the spatial dimensions due to `stride=2`.
  * `ReLU` adds non-linearity after each conv layer.
  * Final output shape after convs: `[B, 64, 2, 2]`.

```python
        self.classifier = nn.Linear(64 * 2 * 2, 10)
```

* A **fully connected layer** that maps flattened conv output (size 256) to 10 output classes (digits 0–9).

```python
    def forward(self, x):
        x = self.net(x)                     # Pass through conv layers
        x = x.flatten(start_dim=1)          # Flatten to shape [B, 256]
        return self.classifier(x)           # Class scores (logits)
```

* **Forward pass logic**:

  1. Extract features.
  2. Flatten features.
  3. Classify with linear layer.


## 🏋️‍♀️ Training the model

In [6]:
# --- Training ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()


for epoch in range(5):  # Single epoch for minimal example
    print(f"Epoch {epoch}")

    # Train
    model.train()
    train_losses = []
    for images, labels in train_loader:
        # Move images to device
        images, labels = images.to(device), labels.to(device)
        # Reset gradients for the new training step
        optimizer.zero_grad()
        # Obtain the predictions from the model
        predictions = model(images)
        # Compute the loss function and backpropagate gradients
        loss = criterion(predictions, labels)
        loss.backward()
        # Update model weights with the gradients
        optimizer.step()
        # Save training loss to show after the epoch
        train_losses.append(loss.item())
    print(f"Train loss: {sum(train_losses) / len(train_losses)}")

    # Validation
    val_losses = []
    with torch.no_grad():
        for images, labels in val_loader:
            # Move images to device
            images, labels = images.to(device), labels.to(device)
            # Obtain the predictions from the model
            predictions = model(images)
            # Compute the loss function
            loss = criterion(predictions, labels)
            # Save validation loss to show after the epoch
            val_losses.append(loss.item())
    print(f"Val loss: {sum(val_losses) / len(val_losses)}")


Epoch 0
Train loss: 0.3728947263123638
Val loss: 0.13161671479458026
Epoch 1
Train loss: 0.120059464773191
Val loss: 0.1023047042648741
Epoch 2
Train loss: 0.08085272296159061
Val loss: 0.06659978079152809
Epoch 3
Train loss: 0.0626897084231832
Val loss: 0.06562400597743068
Epoch 4
Train loss: 0.051872414032514615
Val loss: 0.051669652583125746


```python
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
```

* Selects **GPU** if available, else falls back to **CPU**.

```python
model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
```

* Instantiates the model and moves it to the chosen device.
* Uses **Adam** optimizer with a learning rate of `1e-3`.
* Loss function: **Cross-Entropy**, standard for classification.


🔄 **Epoch Loop**

```python
for epoch in range(5):
    print(f"Epoch {epoch}")
```

* Trains for 5 epochs.


🏋️ **Training Phase**

```python
    # Train
    model.train()
    train_losses = []
    for images, labels in train_loader:
        # Move images to device
        images, labels = images.to(device), labels.to(device)
        # Reset gradients for the new training step
        optimizer.zero_grad()
        # Obtain the predictions from the model
        predictions = model(images)
        # Compute the loss function and backpropagate gradients
        loss = criterion(predictions, y)
        loss.backward()
        # Update model weights with the gradients
        optimizer.step()
        # Save training loss to show after the epoch
        train_losses.append(loss.item())
    print(f"Train loss: {sum(train_losses) / len(train_losses)}")
```

* Sets model to **training mode** (`model.train()`).
* Loads batches of training data, moves to device.
* Clears gradients → forward pass → computes loss → backprop → optimizer step.
* Collects and averages training loss.


🧪 **Validation Phase**

```python
    val_losses = []
    with torch.no_grad():
        for images, labels in val_loader:
            # Move images to device
            images, labels = images.to(device), y.to(device)
            # Obtain the predictions from the model
            predictions = model(images)
            # Compute the loss function
            loss = criterion(predictions, labels)
            # Save validation loss to show after the epoch
            val_losses.append(loss.item())
    print(f"Val loss: {sum(val_losses) / len(val_losses)}")
```

* Disables gradient tracking with `torch.no_grad()` for efficiency. This disables the tracking of gradients necessary to apply the optimization step, but this is not necessary during validation, as we do not train the model.
* Evaluates model on validation data.
* Computes and reports average validation loss. This loss reflects how well the model does on unseen data, giving a more realistic picture of generalization. If the training loss goes down but the validation loss does not, we are overfitting to the training data.
